### 1. 引入工具包

In [1]:
import pandas as pd
import numpy as np

In [2]:
from collections import defaultdict,OrderedDict

In [3]:
from pymining import itemmining, assocrules, perftesting

In [4]:
import orangecontrib.associate.fpgrowth as oaf

In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

### 2. 数据预处理

In [6]:
raw_df = pd.read_csv('./pj/rawData1.CSV', dtype=str)

del raw_df['NO']

In [7]:
raw_df.head()

,Gender,AloneChild,Relationship,NumChildren,Parents,Relatives,Title,ProjectType,ProjectLoc,Position,...,Group_AFC,Group_ACC,Group_LA,Group_NC,WVType,Group_M_GoodTimes,Group_M_Cognitive,Group_M_WorkTeam,Group_M_Instrum,Group_M_Altrui
0,1,1,1,0,0,0,0,NaN,3,1,...,2,2,2,2,22,1,2,2,1,1
1,1,1,1,0,0,0,3,1,1,1,...,3,3,2,3,22,1,1,1,2,1
2,1,1,3,0,1,0,4,1,2,1,...,2,2,2,2,6,1,2,2,2,2
3,1,0,3,2,1,0,4,1,2,3,...,3,2,3,3,19,2,2,3,2,3
4,0,1,1,0,0,1,4,1,2,1,...,3,2,2,3,3,2,1,3,2,1


In [8]:
# 查看有缺失值的columns
# nan_col = pd.DataFrame(raw_df.isnull().any(), columns={'is_nan'})
# nan_col['col'] = nan_col.index
# nan_col_list = list(nan_col[nan_col['is_nan']==True]['col'])
# nan_col_list

#用mode填充缺失值
# pna_df = raw_df.copy()
# for col in nan_col_list:
#     pna_df[col].fillna(pna_df[col].mode()[0], inplace=True)

In [9]:
ConsItem = ['Category_VI', 'Category_DE', 'Category_AB', 'Group_TP', 'Group_IF', 'Group_AFC', 'Group_ACC', 'Group_LA', 'Group_NC']

Supplies = ['Group_J_SAL', 'Group_J_BEN', 'Group_J_ADV', 'Group_J_FAR', 'Group_J_SSU', 'Group_J_CSU', 'Group_J_COW', 'Group_J_COO', 'Group_J_CHA', 'Group_J_CRE', 'Group_J_IST', 'Group_J_CLN', 'Group_J_TRG', 'Group_J_HRS', 'Group_J_BAL', 'Group_J_FRE', 'Group_J_FBK', 'Group_J_IND', 'Group_J_ABI', 'Group_J_INT', 'Group_J_VAR', 'Group_J_FUN', 'Group_J_ENV', 'Group_J_COS', 'Group_J_HLP', 'Group_J_IFL', 'Group_J_IMP']

WorkValues1 = ['Group_SAL', 'Group_BEN', 'Group_ADV', 'Group_FAR', 'Group_SSU', 'Group_CSU', 'Group_COW', 'Group_COO', 'Group_CHA', 'Group_CRE', 'Group_IST', 'Group_CLN', 'Group_TRG', 'Group_HRS', 'Group_BAL', 'Group_FRE', 'Group_FBK', 'Group_IND', 'Group_ABI', 'Group_INT', 'Group_VAR', 'Group_FUN', 'Group_ENV', 'Group_COS', 'Group_HLP', 'Group_IFL', 'Group_IMP']

WorkValues2 = ['Group_M_GoodTimes', 'Group_M_Cognitive', 'Group_M_WorkTeam', 'Group_M_Instrum', 'Group_M_Altrui']

WorkValues3 = ['WVType']

PsJc = ['Gender', 'AloneChild', 'Relationship', 'NumChildren', 'Parents', 'Relatives', 'Title', 'ProjectType', 'ProjectLoc', 'Position', 'DE24_1', 'DE24_2', 'DE24_3', 'DE24_5', 'DE24_6', 'DE24_7', 'DE24_8', 'DE24_9', 'GrowthEnv', 'Edu', 'Contracting', 'Ownership', 'AgeGroup', 'CPMPyrs', 'Org_yrs']

In [10]:
from scipy.sparse import lil_matrix  # other types would convert to LIL anyway

def get_ana_input(raw_df, cols):
    df = raw_df[cols]
    oh_df = pd.get_dummies(df) #one-hot编码
    if 'ProjectType_5' in oh_df.columns.values:
        del oh_df['ProjectType_5'] #删除无效项
    if 'Contracting_4' in oh_df.columns.values:
        del oh_df['Contracting_4']
    oh_df = pd.DataFrame(oh_df, dtype=int) #将属性类型转换为int32
    
    #构造字典记录head的位置
    col_idx = list(range(oh_df.shape[1]))
    col_name = oh_df.columns.values
    ins = zip(col_idx, col_name)
    col_dict = dict((col_idx,col_name) for col_idx, col_name in ins)

    oh_tb = oh_df > .9
    X = lil_matrix(oh_tb.values) #稀疏矩阵
    itme_set = []
    for l in X.rows: #转换成item set
        row = []
        for i in l:
            #row.append(col_dict[i])
            row.append(i)
        itme_set.append(tuple(row))
#     itme_set = tuple(itme_set)
    return itme_set, col_dict

In [11]:
ds_1, col_dic_1 = get_ana_input(raw_df, Supplies + ConsItem)
ds_2, col_dic_2 = get_ana_input(raw_df, Supplies + WorkValues1 + ConsItem)
ds_3, col_dic_3 = get_ana_input(raw_df, Supplies + WorkValues1 + PsJc + ConsItem)
ds_4, col_dic_4 = get_ana_input(raw_df, Supplies + WorkValues2 + ConsItem)
ds_5, col_dic_5 = get_ana_input(raw_df, Supplies + WorkValues2 + PsJc + ConsItem)
ds_6, col_dic_6 = get_ana_input(raw_df, Supplies + WorkValues3 + ConsItem)
ds_7, col_dic_7 = get_ana_input(raw_df, Supplies + WorkValues3 + PsJc + ConsItem)

In [33]:
#生成表单
def rules_to_df(rules, col_dic):  
    return_rules = []
    for i in rules:
        tem_list = []
        k = '';
        for obj in i[0]: 
            k = k + '[' + col_dic[obj] + ']' + ' & '
        
        k = k[:-3]
        flag = False
        v = ''
        for obj in i[1]:
            if col_dic[obj][:-2] not in ConsItem:
                flag = True
                break
            v = v + '[' + col_dic[obj] + ']' + ' & '
        if flag:
            continue
        v = v[:-3]
        tem_list.append(k); tem_list.append(v); tem_list.append(i[2]); tem_list.append(i[3]); tem_list.append(i[4])
        tem_list.append(i[5]); tem_list.append(i[6]); tem_list.append(i[7])
        return_rules.append(tem_list)
    return pd.DataFrame(return_rules, columns=('atecedent','consequent','support','confidence','coverage','strength','lift','leverage'))

In [34]:
from association_rules import rsaa, gen_rules

def get_ass_rules_by_rsaa(ds, col_dic, min_sup = 50, max_sup = 200, r_sup = 0.6, min_conf = 0.6):
    relim_input = itemmining.get_relim_input(ds)
    freq_itemsets = itemmining.relim(relim_input, min_support = max_sup)
    
    rsa = rsaa.RelativeSupportApriori(ds, min_sup, max_sup, r_sup)
    candi_itemsets, helper_itemsets = rsa.get_candidate_itemsets()
    helper_itemsets = defaultdict(int, helper_itemsets.items() | freq_itemsets.items())
    rules = gen_rules.association_rules(candi_itemsets, helper_itemsets, min_conf)

    rules_info = list(gen_rules.rules_stats(rules, defaultdict(int, candi_itemsets.items()| helper_itemsets.items()), len(ds)))    #统计评估信息
    return rules_to_df(rules_info, col_dic)

In [27]:
ds = ds_5
col_dic = col_dic_5
min_sup = 50
max_sup = 200
r_sup = 0.6
min_conf = 0.6

relim_input = itemmining.get_relim_input(ds)
freq_itemsets = itemmining.relim(relim_input, min_support = max_sup)

rsa = rsaa.RelativeSupportApriori(ds, min_sup, max_sup, r_sup)
candi_itemsets, helper_itemsets = rsa.get_candidate_itemsets()
helper_itemsets = defaultdict(int, helper_itemsets.items() | freq_itemsets.items())
rules = gen_rules.association_rules(candi_itemsets, helper_itemsets, min_conf)

rules_info = list(gen_rules.rules_stats(rules, defaultdict(int, candi_itemsets.items()| helper_itemsets.items()), len(ds)))    #统计评估信息

asr = rules_to_df(rules_info, col_dic)

In [28]:
rules_info

[(frozenset({93, 151, 180, 221, 260}),
  frozenset({257}),
  55,
  0.9649122807017544,
  0.05728643216080402,
  13.947368421052632,
  1.2076575085512522,
  0.009504810484583723),
 (frozenset({151, 180, 221, 260}),
  frozenset({93, 257}),
  55,
  0.7432432432432432,
  0.0743718592964824,
  7.6891891891891895,
  1.2996960053199067,
  0.012746142774172369),
 (frozenset({93, 180, 221, 260}),
  frozenset({151, 257}),
  55,
  0.9482758620689655,
  0.05829145728643216,
  11.137931034482758,
  1.4605796946727874,
  0.017430872957753592),
 (frozenset({180, 221, 260}),
  frozenset({93, 151, 257}),
  55,
  0.6962025316455697,
  0.07939698492462312,
  5.949367088607595,
  1.4738755723134933,
  0.017772278477816216),
 (frozenset({93, 151, 180, 260}),
  frozenset({221, 257}),
  55,
  0.9322033898305084,
  0.0592964824120603,
  10.88135593220339,
  1.4447700512170651,
  0.01701674200146461),
 (frozenset({151, 180, 260}),
  frozenset({93, 221, 257}),
  55,
  0.6962025316455697,
  0.07939698492462312,


In [37]:
asr_5 = rules_to_df(rules_info, col_dic)

In [38]:
asr_5

,atecedent,consequent,support,confidence,coverage,strength,lift,leverage
0,[Group_IF_3] & [Position_3] & [Group_J_ABI_4] ...,[Group_TP_3],55,0.964912,0.057286,13.947368,1.207658,0.009505
1,[Group_TP_3] & [Position_3] & [Group_J_ABI_4] ...,[Group_IF_3],55,1.000000,0.055276,15.327273,1.180308,0.008444
2,[Contracting_2] & [Position_3] & [Group_J_ABI_...,[Group_TP_3] & [Group_IF_3],55,0.948276,0.058291,12.758621,1.275047,0.011924
3,[Group_IF_3] & [Position_3] & [Gender_1] & [Gr...,[Group_TP_3],56,0.949153,0.059296,13.474576,1.187933,0.008904
4,[Group_TP_3] & [Position_3] & [Gender_1] & [Gr...,[Group_IF_3],56,0.982456,0.057286,14.789474,1.159601,0.007746
5,[Group_J_HLP_4] & [Position_3] & [Contracting_...,[Group_TP_3] & [Group_IF_3],56,0.903226,0.062312,11.935484,1.214473,0.009939
6,[Group_J_INT_5] & [Group_IF_3] & [Group_AFC_3]...,[Group_TP_3],54,0.981818,0.055276,14.454545,1.228816,0.010106
7,[Group_TP_3] & [Group_J_INT_5] & [Group_AFC_3]...,[Group_IF_3],54,1.000000,0.054271,15.611111,1.180308,0.008291
8,[Group_NC_3] & [Gender_1] & [Group_J_INT_5] & ...,[Group_TP_3] & [Group_IF_3],54,0.981818,0.055276,13.454545,1.320147,0.013161
9,[Group_TP_3] & [Group_J_INT_5] & [Group_IF_3] ...,[Group_AFC_3],54,0.964286,0.056281,9.642857,1.776786,0.023727


In [39]:
asr_7 = get_ass_rules_by_rsaa(ds_7, col_dic_7)

In [40]:
asr_7.head()

,atecedent,consequent,support,confidence,coverage,strength,lift,leverage
0,[Gender_1] & [Category_DE_5] & [Group_IF_3] & ...,[Group_TP_3],94,0.969072,0.097487,8.195876,1.212864,0.016580
1,[Gender_1] & [Category_DE_5] & [Group_TP_3] & ...,[Group_IF_3],94,0.989474,0.095477,8.873684,1.167884,0.013580
2,[Group_NC_3] & [Gender_1] & [Group_AFC_3] & [C...,[Group_TP_3] & [Group_IF_3],94,0.959184,0.098492,7.551020,1.289713,0.021222
3,[Gender_1] & [Category_DE_5] & [Group_TP_3] & ...,[Group_AFC_3],94,0.959184,0.098492,5.510204,1.767385,0.041019
4,[Gender_1] & [Group_NC_3] & [Category_DE_5] & ...,[Group_AFC_3] & [Group_TP_3],94,0.930693,0.101508,4.881188,1.878376,0.044178


In [42]:
asr_5.to_csv('ars_rsaa_5.csv', index=False)
asr_7.to_csv('ars_rsaa_7.csv', index=False)

### test
---

In [25]:
data = [(1, 2, 3), (3, 4), (1, 2, 3, 4, 5, 6, 7), (6, 7), (3, 4, 5, 6), (1, 2), (3, 4, 6, 7), (1, 2),
            (1, 3, 4, 5, 6), (1, 2)]
ds=data
min_sup = 2
max_sup = 4
r_sup = 0.7
min_conf = 0.6

In [26]:
relim_input = itemmining.get_relim_input(ds)
freq_itemsets = itemmining.relim(relim_input, min_support = max_sup)

rsa = rsaa.RelativeSupportApriori(ds, min_sup, max_sup, r_sup)
candi_itemsets, helper_itemsets = rsa.get_candidate_itemsets()

helper_itemsets = defaultdict(int, helper_itemsets.items() | freq_itemsets.items())
rules = gen_rules.association_rules(candi_itemsets, helper_itemsets, min_conf)

rules_info = list(gen_rules.rules_stats(rules, defaultdict(int, helper_itemsets.items() | candi_itemsets.items()), 10))    #统计评估信息
rules_info

[(frozenset({4, 5, 6}),
  frozenset({3}),
  3,
  1.0,
  0.3,
  2.0,
  1.6666666666666667,
  0.12),
 (frozenset({3, 5, 6}),
  frozenset({4}),
  3,
  1.0,
  0.3,
  1.6666666666666667,
  2.0,
  0.15),
 (frozenset({5, 6}),
  frozenset({3, 4}),
  3,
  1.0,
  0.3,
  1.6666666666666667,
  2.0,
  0.15),
 (frozenset({3, 4, 6}), frozenset({5}), 3, 0.75, 0.4, 0.75, 2.5, 0.18),
 (frozenset({4, 6}), frozenset({3, 5}), 3, 0.75, 0.4, 0.75, 2.5, 0.18),
 (frozenset({3, 6}), frozenset({4, 5}), 3, 0.75, 0.4, 0.75, 2.5, 0.18),
 (frozenset({6}), frozenset({3, 4, 5}), 3, 0.6, 0.5, 0.6, 2.0, 0.15),
 (frozenset({3, 4, 5}),
  frozenset({6}),
  3,
  1.0,
  0.3,
  1.6666666666666667,
  2.0,
  0.15),
 (frozenset({4, 5}),
  frozenset({3, 6}),
  3,
  1.0,
  0.3,
  1.3333333333333333,
  2.5,
  0.18),
 (frozenset({3, 5}),
  frozenset({4, 6}),
  3,
  1.0,
  0.3,
  1.3333333333333333,
  2.5,
  0.18),
 (frozenset({5}),
  frozenset({3, 4, 6}),
  3,
  1.0,
  0.3,
  1.3333333333333333,
  2.5,
  0.18),
 (frozenset({3, 4}), 

In [30]:
candi_itemsets

helper_itemsets

freq_itemsets

{frozenset({5}): 3,
 frozenset({7}): 3,
 frozenset({3, 7}): 2,
 frozenset({4, 5}): 3,
 frozenset({6, 7}): 3,
 frozenset({5, 6}): 3,
 frozenset({1, 5}): 2,
 frozenset({3, 5}): 3,
 frozenset({4, 7}): 2,
 frozenset({1, 3, 5}): 2,
 frozenset({4, 6, 7}): 2,
 frozenset({3, 4, 7}): 2,
 frozenset({3, 5, 6}): 3,
 frozenset({1, 4, 5}): 2,
 frozenset({3, 4, 5}): 3,
 frozenset({4, 5, 6}): 3,
 frozenset({3, 6, 7}): 2,
 frozenset({1, 5, 6}): 2,
 frozenset({3, 4, 5, 6}): 3,
 frozenset({1, 4, 5, 6}): 2,
 frozenset({1, 3, 4, 5}): 2,
 frozenset({3, 4, 6, 7}): 2,
 frozenset({1, 3, 5, 6}): 2,
 frozenset({1, 3, 4, 5, 6}): 2}

In [27]:
# ars_1 = get_ass_rules_by_rsaa(ds_1)
# ars_2 = get_ass_rules_by_rsaa(ds_2)
# ars_3 = get_ass_rules_by_rsaa(ds_3)
# ars_4 = get_ass_rules_by_rsaa(ds_4)
asr_5 = get_ass_rules_by_rsaa(ds_5, col_dic_5)
# asr_6 = get_ass_rules_by_rsaa(ds_6, col_dic_6)
asr_7 = get_ass_rules_by_rsaa(ds_7, col_dic_7)

NameError: name 'rule_input' is not defined

In [12]:
ars_1.to_csv('ars_1.csv', index=False)
ars_2.to_csv('ars_2.csv', index=False)
ars_3.to_csv('ars_3.csv', index=False)
ars_4.to_csv('ars_4.csv', index=False)
ars_5.to_csv('ars_5.csv', index=False)
ars_6.to_csv('ars_6.csv', index=False)
ars_7.to_csv('ars_7.csv', index=False)

In [7]:
ars_1 = pd.read_csv('ars_1.csv')
ars_2 = pd.read_csv('ars_2.csv')
ars_3 = pd.read_csv('ars_3.csv')
ars_4 = pd.read_csv('ars_4.csv')
ars_5 = pd.read_csv('ars_5.csv')
ars_6 = pd.read_csv('ars_6.csv')
ars_7 = pd.read_csv('ars_7.csv')

In [22]:
writer = pd.ExcelWriter('ars.xlsx')
for i in range(1, 8):
    df_name = 'ars_' + str(i)
    df = eval(df_name)
    df.to_excel(excel_writer=writer, sheet_name=df_name, encoding="utf-8", index=False)
writer.save()
writer.close()

In [9]:
ars_1.head()

,atecedent,consequent,support,confidence,coverage,strength,lift,leverage
0,[Group_J_HLP_4] & [Group_J_INT_4] & [Group_J_C...,[Group_IF_3],202,0.980583,0.207035,4.092233,1.157390,0.027607
1,[Group_J_HLP_4] & [Group_J_INT_4] & [Group_J_C...,[Group_TP_3] & [Group_IF_3],202,0.922374,0.220101,3.378995,1.240220,0.039322
2,[Group_J_HLP_4] & [Group_J_INT_4] & [Group_J_C...,[Group_TP_3],202,0.961905,0.211055,3.785714,1.203893,0.034383
3,[Group_J_HLP_4] & [Group_J_INT_4] & [Group_J_C...,[Group_IF_3],205,0.990338,0.208040,4.072464,1.168904,0.029771
4,[Group_J_HLP_4] & [Group_J_INT_4] & [Group_J_I...,[Group_TP_3] & [Group_IF_3],205,0.927602,0.222111,3.348416,1.247248,0.040842


In [33]:
df = ars_1
df[~df['consequent'].str.contains('_3')].shape

(333, 8)

In [8]:
def ana_ars(ars_df, min_sup = 50, sup_step = 50, min_conf = 0.3, conf_step = 0.1, min_lift = 1.1):
    df = ars_df
    ars_info = []
    cols = ('support', 'confidence', 'rule_num', 'lift', 'srule_num', 'no2_srule_num', 'no2_srule_lift_avg')
    for sup in range(min_sup, 1000, sup_step):
        one_info = []
        df = df[df['support'] > sup]
        for conf in np.arange(min_conf, 1.0, conf_step):
            df = df[df['confidence'] > conf]
            rule_num = df.shape[0]
            if rule_num == 0:
                break
            df = df[df['lift'] >= min_lift]
            srule_num = df.shape[0]
            df = df[~df['consequent'].str.contains('_2')]
            no2_srule_num = df.shape[0]
            no2_srule_lift_avg = df['lift'].mean()
            one_info = [sup, conf, rule_num, min_lift, srule_num, no2_srule_num, no2_srule_lift_avg]
            ars_info.append(one_info)
    return pd.DataFrame(ars_info, columns=cols)

In [9]:
aa_1 = ana_ars(ars_1)

aa_2 = ana_ars(ars_2)

aa_3 = ana_ars(ars_3)

aa_4 = ana_ars(ars_4)

aa_5 = ana_ars(ars_5)

aa_6 = ana_ars(ars_6)

aa_7 = ana_ars(ars_7)

In [20]:
writer = pd.ExcelWriter('ars_ana.xlsx')
for i in range(1, 8):
    df_name = 'aa_' + str(i)
    df = eval(df_name)
    df.to_excel(excel_writer=writer, sheet_name=df_name, encoding="utf-8", index=False)
writer.save()
writer.close()

### 3. 关联分析

In [445]:
print("------------"+ "minsup: " + str(minsup) + "; minconf: " + str(minconf) + "; rules_num: " + str(tgt_rules_df.shape[0]) + "------------")
tgt_rules_df.head()

------------minsup: 0.3; minconf: 0.6; rules_num: 8529------------


,atecedent,consequent,support,confidence,coverage,strength,lift,leverage
0,[Group_J_COW_3 ] & [Group_IF_2 ] & [Group_J_SS...,[Group_TP_2 ],304,0.910180,0.335678,2.380240,1.139156,0.037322
1,[Group_J_COW_3 ] & [Group_TP_2 ] & [Group_J_SS...,[Group_IF_2 ],304,0.955975,0.319598,2.650943,1.128345,0.034753
2,[Group_J_SSU_3 ] & [Group_J_COW_3 ] & [Group_J...,[Group_TP_2 ] & [Group_IF_2 ],304,0.832877,0.366834,2.027397,1.119882,0.032706
3,[Group_J_COW_3 ] & [Group_TP_2 ] & [Group_J_SS...,[Group_IF_2 ],299,0.949206,0.316583,2.676190,1.120356,0.032282
4,[Group_J_SSU_3 ] & [Group_J_COW_3 ] & [Group_J...,[Group_TP_2 ] & [Group_IF_2 ],299,0.832869,0.360804,2.061281,1.119871,0.032166


In [414]:
print("------------"+ "minsup: " + str(minsup) + "; minconf: " + str(minconf) + "; rules_num: " + str(rules_df.shape[0]) + "------------")
rules_df.head()

------------minsup: 0.3; minconf: 0.6; rules_num: 58032------------


,rules,support,confidence,coverage,strength,lift,leverage
0,[Group_TP_2 ] & [Group_IF_2 ] & [Group_J_SSU_3...,304,0.891496,0.342714,1.976540,1.316080,0.073378
1,[Group_J_CSU_3 ] & [Group_TP_2 ] & [Group_J_CL...,304,0.769620,0.396985,1.278481,1.516380,0.104043
2,[Group_J_SSU_3 ] & [Group_TP_2 ] & [Group_J_CL...,304,0.787565,0.387940,1.347150,1.506975,0.102785
3,[Group_J_COW_3 ] & [Group_TP_2 ] & [Group_IF_2...,304,0.894118,0.341709,1.800000,1.453672,0.095351
4,[Group_J_COW_3 ] & [Group_IF_2 ] & [Group_J_SS...,304,0.910180,0.335678,2.380240,1.139156,0.037322


In [415]:
rules_df[rules_df['lift']>1].sort_values(by='lift', ascending=False)

,rules,support,confidence,coverage,strength,lift,leverage
57931,[Category_WE_2 ] ==> [Category_DE_2 ],311,0.740476,0.422111,0.959524,1.828223,0.141597
57932,[Category_DE_2 ] ==> [Category_WE_2 ],311,0.771712,0.405025,1.042184,1.828223,0.141597
30655,[Group_HLP_3 ] & [Group_CHA_3 ] ==> [Group_...,303,0.740831,0.411055,1.004890,1.793497,0.134730
30659,[Group_COS_3 ] & [Group_IST_3 ] ==> [Group_...,303,0.737226,0.413065,0.995134,1.793497,0.134730
31923,[Group_CRE_3 ] & [Group_CSU_3 ] ==> [Group_...,304,0.785530,0.388945,1.147287,1.760365,0.131968
31921,[Group_SSU_3 ] & [Group_CHA_3 ] ==> [Group_...,304,0.684685,0.446231,0.871622,1.760365,0.131968
7127,[Group_CRE_3 ] & [Group_CLN_3 ] & [Group_IF_2 ...,321,0.829457,0.388945,1.219638,1.748538,0.138109
7118,[Group_CHA_3 ] & [Group_IST_3 ] ==> [Group_...,321,0.680085,0.474372,0.819915,1.748538,0.138109
7098,[Group_CHA_3 ] & [Group_IST_3 ] ==> [Group_...,306,0.648305,0.474372,0.781780,1.748140,0.131615
7084,[Group_CRE_3 ] & [Group_CLN_3 ] & [Group_TP_2 ...,306,0.829268,0.370854,1.279133,1.748140,0.131615


In [416]:
rules_df.to_excel('regular.xlsx')

In [446]:
tgt_rules_df.to_excel('tgt_rules_df.xlsx')